## Taneth Germishuys - Econometrics PS 3 ##

In [1]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
import scipy.stats as stats
from scipy.stats import chi2
from scipy.optimize import minimize
from linearmodels.iv import IV2SLS, IVGMM, compare


## Question 0 ##

# Part A #

In [51]:
# Import Titanic CSV file
data_T = pd.read_csv('TitanicData.csv')

In [52]:
data_T.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
# Check for missing values
columns_to_check = ["Age", "Survived", "Sex", "Pclass"]
missing_values = data_T[columns_to_check].isnull().sum()
print(missing_values)

Age         177
Survived      0
Sex           0
Pclass        0
dtype: int64


In [54]:
# Since "Age" is the only variable with missing observations these observations will be dropped
data_T_cleaned = data_T.dropna(subset=["Age"])

In [55]:
# Confirm there are no longer missing values
columns_to_check = ["Age", "Survived", "Sex", "Pclass"]
missing_values = data_T_cleaned[columns_to_check].isnull().sum()
print(missing_values)

Age         0
Survived    0
Sex         0
Pclass      0
dtype: int64


In [56]:
print(data_T_cleaned['Sex'].unique())

['male' 'female']


In [57]:
# Convert Sex to Male =1 and Female = 0
data_T_cleaned['Sex'] = data_T_cleaned['Sex'].map({'male': 1, 'female': 0})
data_T_cleaned.head()

C:\Users\tanet\AppData\Local\Temp\ipykernel_4268\2940993650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_T_cleaned['Sex'] = data_T_cleaned['Sex'].map({'male': 1, 'female': 0})


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [58]:
# Running the probit model
# Set the variables
X = data_T_cleaned[['Pclass', 'Sex', 'Age']]  # Independent variables
y = data_T_cleaned['Survived']                # Dependent variable (Survived or not)

# Add a constant to the independent variables (for the intercept term)
X = sm.add_constant(X)

# Fit the Probit model
probit_model = sm.Probit(y, X)
probit_result = probit_model.fit()

# Print the summary of the Probit model
print(probit_result.summary())

Optimization terminated successfully.
         Current function value: 0.455180
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                         Probit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Wed, 20 Nov 2024   Pseudo R-squ.:                  0.3261
Time:                        16:34:25   Log-Likelihood:                -325.00
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.167e-68
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.8693      0.270     10.646      0.000       2.341       3.398
Pclass        -0.7206      0.

# Part B #

In [59]:

# Probit model coefficients from the regression output
beta_const = 2.8693
beta_pclass = -0.7206
beta_sex = -1.4838
beta_age = -0.0203

# Attributes for a 55-year-old man traveling in the 3rd class
pclass = 3  
sex = 1  
age = 55  

# Calculate the linear combination of the independent variables (the latent variable)
linear_combination = beta_const + beta_pclass * pclass + beta_sex * sex + beta_age * age

# Convert the latent variable to a probability using the cumulative distribution function of the normal distribution
probability_of_survival = norm.cdf(linear_combination)

probability_of_survival

np.float64(0.029192235792053923)

In [60]:

print(probability_of_survival)

0.029192235792053923


# Part C #

In [61]:
# Calculate the probability of survival for the same 55-year-old man if he were traveling in 1st class
pclass_first_class = 1  # 1st class

# Calculate the linear combination for 1st class
linear_combination_first_class = beta_const + beta_pclass * pclass_first_class + beta_sex * sex + beta_age * age

# Convert the latent variable to a probability for 1st class
probability_of_survival_first_class = norm.cdf(linear_combination_first_class)

print(probability_of_survival_first_class)

0.32577858500766826


In [62]:
# Effect of change from 3rd to 1st class
effect_of_class_change = probability_of_survival_first_class - probability_of_survival

(probability_of_survival_first_class, effect_of_class_change)
print(effect_of_class_change)

0.29658634921561433


## Part D

In [63]:
# Probit model coefficients from part a
beta_const = 2.8693
beta_pclass = -0.7206
beta_sex = -1.4838
beta_age = -0.0203

In [64]:
# Covariance matrix of the coefficients from the Probit model
cov_matrix = probit_result.cov_params().to_numpy()

In [65]:
# Attributes for a 55-year-old man
age = 55
sex = 1

In [66]:
# Define partial derivatives for the probabilities
def compute_gradient(pclass, beta_const, beta_pclass, beta_sex, beta_age):
    # Linear combination for the latent variable
    linear_combination = beta_const + beta_pclass * pclass + beta_sex * sex + beta_age * age

    # Compute the PDF of the standard normal distribution at the linear combination
    phi = norm.pdf(linear_combination)

    # Compute the gradient (partial derivatives w.r.t each coefficient)
    gradient = np.array([
        phi,                    # Partial derivative w.r.t beta_const
        phi * pclass,           # Partial derivative w.r.t beta_pclass
        phi * sex,              # Partial derivative w.r.t beta_sex
        phi * age               # Partial derivative w.r.t beta_age
    ])
    return gradient

In [67]:
# Compute the gradient for 1st class and 3rd class
gradient_1st_class = compute_gradient(1, beta_const, beta_pclass, beta_sex, beta_age)
gradient_3rd_class = compute_gradient(3, beta_const, beta_pclass, beta_sex, beta_age)

In [68]:
# Compute the difference in gradients
gradient_difference = gradient_1st_class - gradient_3rd_class

In [69]:
# Delta Method: Compute the variance of the effect
variance_delta_p = gradient_difference.T @ cov_matrix @ gradient_difference

In [70]:

# Standard error of the effect
std_error_delta_p = np.sqrt(variance_delta_p)

In [71]:
# Print the result
print(f"Standard Error of the Effect (Delta Method): {std_error_delta_p}")

Standard Error of the Effect (Delta Method): 0.040779280942521055


## Question 1 ##

# Part D #

In [72]:
# Import the file
data_SP = pd.read_excel('SP500Index.xlsx')

In [73]:
data_SP.head()

,Date of Observation,Level of the S&P 500 Index
0,19600129,55.61
1,19600229,56.12
2,19600331,55.34
3,19600429,54.37
4,19600531,55.83


In [74]:
data_SP['Date']=data_SP['Date of Observation'].astype(str)
data_SP.head()

,Date of Observation,Level of the S&P 500 Index,Date
0,19600129,55.61,19600129
1,19600229,56.12,19600229
2,19600331,55.34,19600331
3,19600429,54.37,19600429
4,19600531,55.83,19600531


In [75]:
# Identify SP_0 (January 1960)
SP0 = data_SP.loc[data_SP['Date'].str.startswith("196001"), 'Level of the S&P 500 Index'].iloc[0]
print(SP0)

55.61


In [76]:
# Calculate x_t = log(SP_t / SP_0)
data_SP['x_t'] = np.log(data_SP['Level of the S&P 500 Index'] / SP0)

In [77]:
# Define the log-likelihood function
def log_likelihood(params, x_t):
    delta, sigma2 = params  # Parameters to estimate
    T = len(x_t)
    residuals = x_t[1:] - delta - x_t[:-1]  # Differences x_t - delta - x_{t-1}
    logL = -0.5 * T * np.log(2 * np.pi * sigma2) - (1 / (2 * sigma2)) * np.sum(residuals**2)
    return -logL  # Negate for minimization


In [78]:
# Optimize to find MLEs
x_t = data_SP['x_t'].to_numpy()  # Convert x_t to a NumPy array
initial_guess = [0, 1]  # Initial guesses for delta and sigma^2
result = minimize(log_likelihood, initial_guess, args=(x_t,), bounds=[(None, None), (1e-6, None)])



In [79]:
# Extract MLEs
delta_MLE, sigma2_MLE = result.x
print(f"MLE for delta: {delta_MLE}")
print(f"MLE for sigma^2: {sigma2_MLE}")

# Step 6: Interpret results
sigma_MLE = np.sqrt(sigma2_MLE)  # Standard deviation
print(f"Standard deviation (sigma): {sigma_MLE}")

MLE for delta: 0.0055483897567819365
MLE for sigma^2: 0.0017766419802605514
Standard deviation (sigma): 0.04215023108193538


## Question 5 ##

## Part a ##

In [80]:
# Import the Hall data file - PS4Data #
data_H = pd.read_excel('PS4data.xls')

In [81]:
# Check the imported data #
data_H.head()

,year,quarter,real consumption of nondurables,real consumption of services,cpi,real disposable income,population,equally weighted average return on the New York Stock Exchange
0,1948,1,442.3,463.8,23.533333,1055.3,102690.67,0.01414
1,1948,2,446.9,470.5,23.933333,1087.7,102915.33,0.11755
2,1948,3,443.3,476.6,24.466667,1107.1,103249.00,-0.09565
3,1948,4,449.6,480.2,24.233333,1109.8,103417.67,-0.04563
4,1949,1,451.8,482.6,23.866667,1087.8,103584.33,0.03338


In [82]:
# Adding required columns #
data_H['consum nondurable per capita'] = data_H['real consumption of nondurables'] / data_H['population']
data_H['consump services per capita'] = data_H['real consumption of services'] / data_H['population']

In [83]:
data_H['consumption'] = data_H['consum nondurable per capita'] + data_H['consump services per capita']

In [84]:
data_H.head()

,year,quarter,real consumption of nondurables,real consumption of services,cpi,real disposable income,population,equally weighted average return on the New York Stock Exchange,consum nondurable per capita,consump services per capita,consumption
0,1948,1,442.3,463.8,23.533333,1055.3,102690.67,0.01414,0.004307,0.004516,0.008824
1,1948,2,446.9,470.5,23.933333,1087.7,102915.33,0.11755,0.004342,0.004572,0.008914
2,1948,3,443.3,476.6,24.466667,1107.1,103249.00,-0.09565,0.004294,0.004616,0.008910
3,1948,4,449.6,480.2,24.233333,1109.8,103417.67,-0.04563,0.004347,0.004643,0.008991
4,1949,1,451.8,482.6,23.866667,1087.8,103584.33,0.03338,0.004362,0.004659,0.009021


In [85]:
# Create lagged consumption variables (Ct-1, Ct-2, Ct-3, Ct-4)
data_H['Ct-1'] = data_H['consumption'].shift(1)
data_H['Ct-2'] = data_H['consumption'].shift(2)
data_H['Ct-3'] = data_H['consumption'].shift(3)
data_H['Ct-4'] = data_H['consumption'].shift(4)

In [86]:
# Drop na
data_H = data_H.dropna()

In [87]:
# Run the regression
X = data_H[['Ct-1', 'Ct-2', 'Ct-3', 'Ct-4']]
Y = data_H['consumption']

# Add a constant
X = sm.add_constant(X)

In [88]:
# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            consumption   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.094e+05
Date:                Wed, 20 Nov 2024   Prob (F-statistic):               0.00
Time:                        16:34:27   Log-Likelihood:                 1745.1
No. Observations:                 216   AIC:                            -3480.
Df Residuals:                     211   BIC:                            -3463.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.439e-05   1.92e-05      1.267      0.2

In [89]:
# Run F test to test the hypothesis

# Hypothesis
hypothesis = 'Ct-2 = Ct-3 = Ct-4 = 0'
f_test = model.f_test(hypothesis)
print(f_test)

<F test: F=8.409996895528458, p=2.6364145736178056e-05, df_denom=211, df_num=3>


## Part c ##

In [90]:
# Import the Hall data file - PS4Data #
data_H2 = pd.read_excel('PS4data.xls')

# Create disposable income measure and consumption
data_H2['income per capita'] = data_H2['real disposable income'] / data_H2['population']
data_H2['consum nondurable per capita'] = data_H2['real consumption of nondurables'] / data_H2['population']
data_H2['consump services per capita'] = data_H2['real consumption of services'] / data_H2['population']
data_H2['consumption'] = data_H2['consum nondurable per capita'] + data_H2['consump services per capita']

# Create log variables
data_H2['ln_income'] = np.log(data_H2['income per capita'])
data_H2['ln_consumption'] = np.log(data_H2['consumption'])

# Create lagged variables
data_H2['log(Ct/Ct-1)'] = data_H2['ln_consumption'] - data_H2['ln_consumption'].shift(1)
data_H2['log(Ct-2/Ct-3)'] = data_H2['ln_consumption'].shift(2) - data_H2['ln_consumption'].shift(3)
data_H2['log(Ct-3/Ct-4)'] = data_H2['ln_consumption'].shift(3) - data_H2['ln_consumption'].shift(4)
data_H2['log(Ct-4/Ct-5)'] = data_H2['ln_consumption'].shift(4) - data_H2['ln_consumption'].shift(5)
data_H2['log(Ct-5/Ct-6)'] = data_H2['ln_consumption'].shift(5) - data_H2['ln_consumption'].shift(6)

data_H2['log(Yt/Yt-1)'] = data_H2['ln_income'] - data_H2['ln_income'].shift(1)

# Show data
data_H2.head()


,year,quarter,real consumption of nondurables,real consumption of services,cpi,real disposable income,population,equally weighted average return on the New York Stock Exchange,income per capita,consum nondurable per capita,consump services per capita,consumption,ln_income,ln_consumption,log(Ct/Ct-1),log(Ct-2/Ct-3),log(Ct-3/Ct-4),log(Ct-4/Ct-5),log(Ct-5/Ct-6),log(Yt/Yt-1)
0,1948,1,442.3,463.8,23.533333,1055.3,102690.67,0.01414,0.010276,0.004307,0.004516,0.008824,-4.577896,-4.730327,NaN,NaN,NaN,NaN,NaN,NaN
1,1948,2,446.9,470.5,23.933333,1087.7,102915.33,0.11755,0.010569,0.004342,0.004572,0.008914,-4.549841,-4.720118,0.010209,NaN,NaN,NaN,NaN,0.028055
2,1948,3,443.3,476.6,24.466667,1107.1,103249.00,-0.09565,0.010723,0.004294,0.004616,0.008910,-4.535400,-4.720634,-0.000516,NaN,NaN,NaN,NaN,0.014442
3,1948,4,449.6,480.2,24.233333,1109.8,103417.67,-0.04563,0.010731,0.004347,0.004643,0.008991,-4.534596,-4.711562,0.009072,0.010209,NaN,NaN,NaN,0.000804
4,1949,1,451.8,482.6,23.866667,1087.8,103584.33,0.03338,0.010502,0.004362,0.004659,0.009021,-4.556229,-4.708237,0.003325,-0.000516,0.010209,NaN,NaN,-0.021633


In [91]:
# Drop rows with NaN caused by lagging
data_H2 = data_H2.dropna()



In [92]:
# Stage 1 - Regress log(Yt/Yt-1) on instruments
X_stage12 = data_H2[['log(Ct-2/Ct-3)', 'log(Ct-3/Ct-4)', 'log(Ct-4/Ct-5)', 'log(Ct-5/Ct-6)']]
X_stage12 = sm.add_constant(X_stage12)  # Add a constant
y_stage12 = data_H2['log(Yt/Yt-1)']  # Endogenous variable

# Fit Stage 1 regression
stage1_model2 = sm.OLS(y_stage12, X_stage12).fit()

# Get fitted values from Stage 1
data_H2['fitted_X'] = stage1_model2.fittedvalues

print(stage1_model2.summary())

                            OLS Regression Results                            
Dep. Variable:           log(Yt/Yt-1)   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     2.509
Date:                Wed, 20 Nov 2024   Prob (F-statistic):             0.0430
Time:                        16:34:27   Log-Likelihood:                 687.27
No. Observations:                 214   AIC:                            -1365.
Df Residuals:                     209   BIC:                            -1348.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0040      0.001      3.

In [93]:
# Stage 2 - Regress log(Ct/Ct-1) on fitted_X
Y_stage22 = data_H2['log(Ct/Ct-1)']
X_stage22 = data_H2['fitted_X']
X_stage22 = sm.add_constant(X_stage22)  # Add a constant

# Fit Stage 2 regression
stage2_model2 = sm.OLS(Y_stage22, X_stage22).fit(cov_type='HC0')

# Print Stage 2 regression summary
print(stage2_model2.summary())

                            OLS Regression Results                            
Dep. Variable:           log(Ct/Ct-1)   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     4.638
Date:                Wed, 20 Nov 2024   Prob (F-statistic):             0.0324
Time:                        16:34:27   Log-Likelihood:                 825.70
No. Observations:                 214   AIC:                            -1647.
Df Residuals:                     212   BIC:                            -1641.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.001      2.179      0.0

## Part D ##

In [94]:
# Rerun Stage 1 and store residuals
X_stage12 = data_H2[['log(Ct-2/Ct-3)', 'log(Ct-3/Ct-4)', 'log(Ct-4/Ct-5)', 'log(Ct-5/Ct-6)']]
X_stage12 = sm.add_constant(X_stage12)  # Add a constant
y_stage12 = data_H2['log(Yt/Yt-1)']  # Endogenous variable

# Fit Stage 1 regression
stage1_model22 = sm.OLS(y_stage12, X_stage12).fit()

# Save residuals
data_H2['residuals'] = stage1_model22.resid

# Print
print(stage1_model22.summary())

                            OLS Regression Results                            
Dep. Variable:           log(Yt/Yt-1)   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     2.509
Date:                Wed, 20 Nov 2024   Prob (F-statistic):             0.0430
Time:                        16:34:27   Log-Likelihood:                 687.27
No. Observations:                 214   AIC:                            -1365.
Df Residuals:                     209   BIC:                            -1348.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0040      0.001      3.

In [95]:
# Control Function Regression (add residuals to second-stage regression)
X_stage22 = data_H2[['log(Yt/Yt-1)', 'residuals']]  # Include endogenous variable and residuals
X_stage22 = sm.add_constant(X_stage22)  # Add a constant
Y_stage22 = data_H2['log(Ct/Ct-1)']

# Fit the control function regression
control_function_model2 = sm.OLS(Y_stage22, X_stage22).fit()

# Print the summary of the regression
print(control_function_model2.summary())

                            OLS Regression Results                            
Dep. Variable:           log(Ct/Ct-1)   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     29.64
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           4.53e-12
Time:                        16:34:27   Log-Likelihood:                 848.67
No. Observations:                 214   AIC:                            -1691.
Df Residuals:                     211   BIC:                            -1681.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0026      0.001      3.107   

In [96]:
# Check the p-value of the coefficient of 'residuals'
if control_function_model2.pvalues['residuals'] < 0.05:
    print("Reject H0: log(Yt/Yt-1) is endogenous.")
else:
    print("Fail to reject H0: log(Yt/Yt-1) might be exogenous.")

Fail to reject H0: log(Yt/Yt-1) might be exogenous.


In [97]:
# Wu Hausman Testing using linearmodels
# Define dependent variable, endogenous variable, and instruments
Y = data_H2['log(Ct/Ct-1)']  # Dependent variable
endog = data_H2['log(Yt/Yt-1)']  # Endogenous variable
instruments = data_H2[['log(Ct-2/Ct-3)', 'log(Ct-3/Ct-4)', 'log(Ct-4/Ct-5)', 'log(Ct-5/Ct-6)']]

# Add a constant to instruments
exog = sm.add_constant(pd.DataFrame({'constant':1}, index=data_H2.index))


In [98]:
# 1. Fit the OLS Model using linearmodels
ols_model = IV2SLS(dependent=Y, exog=endog, endog=None, instruments=None).fit()

# 2. Fit the 2SLS Model using linearmodels
iv_model = IV2SLS(dependent=Y, exog=exog, endog=endog, instruments=instruments).fit()

In [99]:
# Using Wooldridge regression test #
print(iv_model.wooldridge_regression)
print(iv_model.wooldridge_overid)

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 1.1399
P-value: 0.2857
Distributed: chi2(1)
Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 5.0870
P-value: 0.1655
Distributed: chi2(3)


In [100]:
print(iv_model)

                          IV-2SLS Estimation Summary                          
Dep. Variable:           log(Ct/Ct-1)   R-squared:                      0.0685
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0641
No. Observations:                 214   F-statistic:                    4.6529
Date:                Wed, Nov 20 2024   P-value (F-stat)                0.0310
Time:                        16:34:27   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
constant         0.0026     0.0012     2.2579     0.0240      0.0003      0.0049
log(Yt/Yt-1)     0.4369     0.2026     2.157

In [101]:
# This is not actually used. It would have been used for the Wu-Hausman test statistic (manual calculation)
# Step 1: Difference in coefficients
beta_diff = ols_model.params - iv_model.params

# Step 2: Variance of the difference
cov_diff = ols_model.cov + iv_model.cov
test_stat = beta_diff.T @ np.linalg.inv(cov_diff) @ beta_diff

# Step 3: Degrees of freedom
df = len(beta_diff)  # Number of coefficients compared

# Step 4: Compute the p-value
p_value = 1 - chi2.cdf(test_stat, df)

# Output the results
print(f"Wu-Hausman Test Statistic: {test_stat:.4f}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value:.4f}")

Wu-Hausman Test Statistic: nan
Degrees of Freedom: 2
P-value: nan


In [102]:
# Sargan test for overidentification
# Extract residuals from the 2SLS regression
residuals2 = stage2_model2.resid  # Residuals from the 2SLS second stage

# Regress residuals on all instruments
instruments = data_H2[['log(Ct-2/Ct-3)', 'log(Ct-3/Ct-4)', 'log(Ct-4/Ct-5)', 'log(Ct-5/Ct-6)']]
instruments = sm.add_constant(instruments)  # Add a constant

# Fit the regression
sargan_model = sm.OLS(residuals2, instruments).fit()

print(sargan_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.884
Date:                Wed, 20 Nov 2024   Prob (F-statistic):              0.114
Time:                        16:34:27   Log-Likelihood:                 829.49
No. Observations:                 214   AIC:                            -1649.
Df Residuals:                     209   BIC:                            -1632.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.0007      0.001     -1.

In [103]:
# Calculate the Sargan statistic
n = len(residuals2)  # Number of observations
R_squared = sargan_model.rsquared  # R-squared from the regression
Sargan_stat = n * R_squared  # Sargan statistic
print(Sargan_stat)
print(R_squared)
print(n)

7.449032022963289
0.03480856085496864
214


In [104]:
# Degrees of freedom for the chi-square test
l = 4
k = 1  # Number of endogenous regressors
df = l - k  # Degrees of freedom
print(l)
print(df)

4
3


In [105]:
# Chi-square
chi_square_critical = chi2.ppf(1 - 0.05, df)
print(chi_square_critical)

7.814727903251179
